# DMSP-OLS annual composites in Google Earth Engine (5 min)

This module looks at the various ways to transform and aggregate nighttime lights data.

Two common processes are compositing and mosaicing.

<div class="alert alert-info">
In the context of remote sensing, the process of <b>making a composite</b> generally refers to taking a collection of spatially overlapping images and applying an aggregate function, also known as a "reducer" function, to these images to create a single composite image. For example, you may want to create an annual composite for a series of images by applying a reducer function, such as calculating the median, to the images to get a single image for the year that contains the median values per pixel.
</div>


<div class="alert alert-info">
<b>Mosaicing</b> refers to the process of assembling different spatially located images from the same time period together to create a seamless single image. 
</div>


In this tutorial, we'll identify and visualize annual composites in the DMSP-OLS nighttime lights series.

The DMSP-OLS image collection in Google Earth Engine, <a href="https://eogdata.mines.edu/dmsp/downloadV4composites.html">sourced by the Earth Observation Group</a>, Payne Institute for Public Policy, Colorado School of Mines, has been processed as a series of annual composites per year, per satellite (some years include two satellite sensors), so we dont have to compose them ourselves. See Elvidget et al. (1997) {cite}`elvidge1997mapping` for the methdology.

**Prerequisites:**
- Make sure you have Python, Jupyter notebooks, and geemap installed and are familiar with these packages
- If not, you'll want to review: 
    - {doc}`mod1_3_getting_started_with_Python`
    - {doc}`mod1_4_introduction_to_Jupyter_notebooks`
    - {doc}`mod1_5_introduction_to_GEE` (you'll also need to have your GEE account created as shown in this tutorial)
- You may also want to try this exercise to get familiar with GEE and the Python `geemap` library:
    - {doc}`mod1_7_practical_exercise-image_visualization`

**Our tasks in this exercise:**
1. Load and inspect the DMSP-OLS nighttime lights Image Collection in GEE using `geemap`
2. Search the reference table of DMSP-OLS satellites by year for particular annual composite (from 1996).
3. Add the selected annual composite to a map.
4. Add a 2nd annual composite from another year (2010) and create a slider panel to view and compare both.


## Load and inspect the DMSP-OLS nighttime lights Image Collection

### Initialize map object
First, let's import `geemap` and initialize a geemap object centered on the greater Washington, DC area.
We'll also add the default satellite (daytime) basemap.

In [19]:
# import geemap and ee for our Python session
import geemap, ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# set our initial map parameters for Washington, DC
center_lat = 38.9072
center_lon = -77.0369
zoomlevel=10

# initialize our map
dmspMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
dmspMap.add_basemap('SATELLITE')

In [15]:
# display our map
dmspMap

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### Get DMSP-OLS Image Collection and review meta-data and description

Recall the image collection is located at: `NOAA/DMSP-OLS/NIGHTTIME_LIGHTS`

In [16]:
dmsp = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")

**What is the total number of images in this collection?** 
(It spans 1992-2013, but some years contain multiple images due to satellite overlap.)

In the Google Earth Engine Editor, we can just call the `.size()` method on our collection.

With `geemap` we can do the same thing, however, `.size()` will produce a size "object", so we have to add the extra step of using the `.getInfo()` method so it prints out to our notebook.

In the GEE editor (in JavaScript):
```
print(dmsp.size());
```

In our Python notebook:
```
print(dmsp.size().getInfo())
```

In [57]:
print(dmsp.size().getInfo())

35


**TIP:** We can use Python's functional string method by adding `f` and brackets to dynamically print our collection size in a sentence:

In [58]:
print(f"There are {dmsp.size().getInfo()} images in this collection.")

There are 35 images in this collection.


**What is the date range of our collection?**

GEE has a set of methods called "Reducers," which do a range of functions, such as get the sum or avg value of a collection. They are quite handy. The function `Reducer.minMax()` can be used to get a date range.


In the GEE editor (in JavaScript):
```
var imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]);
var start = ee.Date(imgrange.get('min'));
var end = ee.Date(imgrange.get('max'));
print('Date range: ', start, end);
```

In our Python notebook:
```
imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start
```

var range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print('Date range: ', ee.Date(range.get('min')), ee.Date(range.get('max')))

In [59]:
imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])

In [69]:
datetime.fromtimestamp(ee.Date(imgrange.get('min')).getInfo()['value']/1000)

datetime.datetime(1991, 12, 31, 19, 0)

In [63]:
from datetime import datetime


In [70]:
datetime.fromtimestamp(694224000000/1000)

datetime.datetime(1991, 12, 31, 19, 0)

### Get DMSP-OLS image ID for 1992

**Recall from** {doc}`mod1_2_introduction_to_nighttime_light_data`:<br><a href="https://ngdc.noaa.gov/eog/dmsp/downloadV4composites.html" class="alert-link">NOAA`s 
National Center for Environmental Information</a> has the reference table of DMSP satellites: 


**DMSP-OLS satellites by year**

|      | F10     | F12     | F14     | F15     | F16     | F18     |
|------|---------|---------|---------|---------|---------|---------|
| 1992 | F101992 |         |         |         |         |         |
| 1993 | F101993 |         |         |         |         |         |
| 1994 | F101994 | F121994 |         |         |         |         |
| 1995 |         | F121995 |         |         |         |         |
| 1996 |         | F121996 |         |         |         |         |
| 1997 |         | F121997 | F141997 |         |         |         |
| 1998 |         | F121998 | F141998 |         |         |         |
| 1999 |         | F121999 | F141999 |         |         |         |
| 2000 |         |         | F142000 | F152000 |         |         |
| 2001 |         |         | F142001 | F152001 |         |         |
| 2002 |         |         | F142002 | F152002 |         |         |
| 2003 |         |         | F142003 | F152003 |         |         |
| 2004 |         |         |         | F152004 | F162004 |         |
| 2005 |         |         |         | F152005 | F162005 |         |
| 2006 |         |         |         | F152006 | F162006 |         |
| 2007 |         |         |         | F152007 | F162007 |         |
| 2008 |         |         |         |         | F162008 |         |
| 2009 |         |         |         |         | F162009 |         |
| 2010 |         |         |         |         |         | F182010 |
| 2011 |         |         |         |         |         | F182011 |
| 2012 |         |         |         |         |         | F182012 |
| 2013 |         |         |         |         |         | F182013 |

In [6]:
dmsp92id = "NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F101992"

## Create (and adjust) the Nigeria 1992 nighttime lights layer

Now that we know what Image we're looking for, we can query it via the Python API and add it as a layer to that map object we created earlier.

In [7]:
# import the module ee, which was automatially installed when we installed geemap
import ee

# create an ee object for our 1992 image
# note that for DMSP, there is only one band, so we dont need to worry about selecting a band.
dmsp92 = ee.Image(dmsp92id)

# ladd this image as a layer to our map object and call the layer: "DMSP NTL 1992"
myFirstMap.addLayer(dmsp92, name='DMSP NTL 1992')

In [8]:
# re-display our map
myFirstMap

Map(center=[9.0, 7.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Voila! We have a nighttime layer from the 1992 DMSP-OLS composite.

**Pro tip:** We didn't need to re-type `myFirstMap` to display it again. Because we created an interactive object, we could just scroll back up to the cell with `myFirstMap` displayed above the first time and re-execute that cell to show our map with this new layer. This is handy if you want to save space and avoid duplicating displays.

### Changing opacity
You may notice it's quite dark; however. You can always toggle the layer off, but if you want to visualize the nighttime lights over the basemap, you'll want to change the opacity of your nighttime lights layer. Fortunately, this is very easy for us to do.

Our `.addLayer` function allows for other visual parameters, like `opacity`. Let's give this layer an opacity of 75%:

In [9]:
myFirstMap.addLayer(dmsp92, name='DMSP NTL 1992', opacity=0.75)

**Note:** by giving our updated layer the same name (`DMSP NTL 1992`) we are over-writing the previous layer. If you wanted to create a new layer, this time with 50% opacity, but keep the previous layer, just change the layer name when you update, like `DMSP NTL 1992 opc. 50%`. Then you'd have both layers and you can toggle them on/off.

## Creating a mask
Another important step to "clean" your image will be to create a mask that filters out zero or negative values, which can happen after preprocessing for noisy and low-light pixels.

This can be done when adding (or updating) a layer. The ee Image object we created, `dmsp92`, has a built-in method called `.mask()` and when we call that and pass the Image itself as an argument, we get the mask.

<div class="alert alert-success">
<a href="https://developers.google.com/earth-engine/tutorials/tutorial_api_05#masking">This documentation</a> gives  more info on the GEE API .mask() call and we'll get into more detail on these data processing steps later.</div>

This time, let's change the name so that we create a new layer. Then we'll have a masked and non-masked 1992 layer:

In [10]:
myFirstMap.addLayer(dmsp92.mask(dmsp92), name='DMSP NTL 1992 masked', opacity=0.75)

## Change the basemap

Go to the cell with our map object and re-run the cell, updating it. You should now have two layers (intial 1992 and masked 1992) and you can toggle between them. You can imagine that the masked layer makes it easier to inspect the underlying basemap layer.

The default basemap is Open Street Maps (OSM). But if you want to analyze nighttime lights according to land build-up as seen in daylight images (such as from LANDSAT), you can change the basemap (if you're more advanced you can search GEE for your own layers of course).

There are a few dozen options to choose from for geemap basemaps. While there's not documentation yet, you can see the options in the <a href="https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py">source code itself.</a>. Some of these have also been added to `ipyleaflet`'s library.

Navigate to that source code link and review the options. Let's choos the default "SATELLITE" basemap which appears to be of the Google maps daytime satellite view.

Because we're adding a basemap, we'll need to re-initialize our map object, add our new basemap to it, and then add the layers we created initially. If we've used variables for our parameters, this is simple to do.

In [11]:
# initial map object (this overwrites our previous object saved at this variable name)
# centered on Abuja
myFirstMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# add our alternate basemap
myFirstMap.add_basemap("SATELLITE")

# add our 1992 (and remember to create a mask and change opacity to 75%)
myFirstMap.addLayer(dmsp92.mask(dmsp92), name='DMSP NTL 1992', opacity=0.75)

In [12]:
# display our map
myFirstMap

Map(center=[9.0, 7.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Visual inspection

Now you can look at nighttime layer and compare it to our satellite view basemap.

Take a look around. Interact with the map you just created:
- Toggle the satellte basemap off to compare nighttime lights to the road network as well as the satellite view.
- Navigate to Abuja and zoom in. 
- Can you see where the overlap of the nighttime lights are with the roads and "built up" areas? 
- How well do they overlap? 
- Are there any surprises?
- What about other parts of Nigeria?

## Add a layer for DMSP-OLS 2013

Now let's look at nighttime lights for 2013. 

To do this, we can just add a new layer to our object.

Can you do this on your own?

In [11]:
# your code here


<br><br><br><br><br><br><br><br><br><br><br><br>

Need some hints?

In [12]:
# find the Image ID for DMSP-OLS 2013 and set it as a new variable (hint: the satellite's name is "F18")


# create the ee object


# add this image object as a new layer in your map
# name it "DMSP NTL 2013", create a mask, and give it an opacity of 75%.

You should have been able to add a new layer and when refreshing the cell for `myFirstMap` above, see your new layer. If not, scroll down to see the code...

<br><br><br><br><br><br><br><br><br><br><br><br>

In [13]:
# find the Image ID and set it as a new variable (hint: the satellite's name is "F18")
dmsp2013id = "NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182013"

# create the ee object
dmsp2013 = ee.Image(dmsp2013id)

# add this image as a new layer in your map object, myFirstMap
# name it "DMSP NTL 2013" and give it an opacity of 75%.

myFirstMap.addLayer(dmsp2013.mask(dmsp2013), name='DMSP NTL 2013', opacity=0.75)

In [17]:
# display the map object again...
myFirstMap

Map(bottom=8056.0, center=[9.0, 7.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

Now that you have both years, you can toggle back and forth and compare the differences.

Do you see any major changes?

Zoom in for a closer look at Abuja: do you see the growth from 1992 to 2013?

## Create a split planel view

We've added our 2013 layer to compare with 1992, but it's kind of annoying to toggle each layer. It's also hard to truly compare. If we create a split panel view with a slider, we can more easily see the difference.

There is a built-in method in `geemap` for this, which makes it simple to do.

We've already created our 1992 and 2013 DMSP image objects and saved those as variables, so no need to re-create. We just need to generate a tile layer with each. But remember to mask them and let's again set opacity to 75%:

In [15]:
# generate tile layers from the ee image objects, masking and changing opacity to 75%
dmsp92_tile = geemap.ee_tile_layer(dmsp92.mask(dmsp92), {}, 'DMSP NTL 1992', opacity=0.75)
dmsp2013_tile = geemap.ee_tile_layer(dmsp2013.mask(dmsp2013), {}, 'DMSP NTL 2013', opacity=0.75)

If you want to create a new map object you can, just as before and center it using the parameters for Nigeria we set before:

`newMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)`

But you can also just alter the initial object we created, which is what we'll do. We can call the object's `.split_map()` method and set the left and panels with our 1992 and 2013 tile layers.

In [16]:
myFirstMap.split_map(left_layer=dmsp92_tile, right_layer=dmsp2013_tile)

Now when you refresh your map object above, you'll see a slider and the 1992 nighttime lights layer on the left with 2013 on the right. 

Slide it over Abuja: can you see the difference in the distribution of lights in 2013 compared to 1992? 

Look at some other regions around Nigeria.

## On your own...
Our primary objective was to get a feel for using `geemap`, GEE, and Jupyter notebooks. Hopefully, you've now done that.

Try comparing other years of DMSP-OLS data by creating new layers. Keep exploring Nigeria or try navigating (and centering your initial map object) on other parts of the world.

This interactive viewer is a simple way to view changes in an area, but the real power is in conducting analysis. You can image that quantifying the difference in nighttime lights from 1992 to 2013 or calculating the slope of change across a time series could reveal areas of high or low growth and other patterns. 

We'll learn how to do this as well as deal with important issues of calibration. These satellite detectors, like all sensors, can change over time or have inherent biases (known as "instrument bias"), so it's important understand how to address these in order to do a fair comparison.